In [ ]:
!pip install opencv-python-headless numpy
!wget https://pjreddie.com/media/files/yolov3.weights -O yolov3.weights
!wget https://github.com/pjreddie/darknet/raw/master/cfg/yolov3.cfg -O yolov3.cfg
!wget https://github.com/pjreddie/darknet/raw/master/data/coco.names -O coco.names

In [ ]:
import cv2
import numpy as np
from PIL import Image
from google.colab.patches import cv2_imshow
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print(f"File which is uploaded: {file_name}")

#Read the uploaded image with OpenCV
image = cv2.imread(file_name)

#Check if the image is uploaded successfully
if image is None:

  raise ValueError("Image didn't upload. Please check the file which is uploaded.") #If you get that error it's probably bc of the image format is not support bt google colab so you can use the code which is i gave under of code in comment line
else:
  print("Image uploaded succesfully:", image.shape)


In [ ]:
# That code block is show your image is valid or invalid display format
print(f"File name: {file_name}")
!ls -l
from PIL import Image

try:
    img_test = Image.open(file_name)
    img_test.verify()  # verify that it is, in fact an image
    print("Image is valid foramt")
except Exception as e:
    raise ValueError(f"Image is invalid foramt: {e}")

In [ ]:
# Show the image
cv2_imshow(image) #original
cv2.imread(file_name) #processed

In [ ]:
# Downloaded with pillow
img = Image.open(file_name)
image = np.array(img)

print("Image uploaded Pillow succesfully:", image.shape)

In [ ]:
weights_path = "yolov3.weights"
config_path = "yolov3.cfg"
names_path = "coco.names"

with open(names_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Upload YOLO network
net = cv2.dnn.readNet(weights_path, config_path)

# Processed image
height, width, _ = image.shape

blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

outs = net.forward(output_layers)

# Object detection
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:  #Confidence threshold
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)
cv2_imshow(image) #original

In [ ]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)
# Draw the objects which is detected
if len(indices) == 0:
    print("None of the object detected.")
else:
    for i in indices.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = confidences[i]
        color = (0, 255, 0)  #Color of box is green
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
# Show the objects which is detected
cv2_imshow(image) #processed